In [1]:
import os
os.chdir("../")
%pwd

'/home/izam/coding/dlproject'

In [2]:
import tensorflow as tf

2023-12-02 09:13:25.802358: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-02 09:13:25.802433: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-02 09:13:25.804950: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-02 09:13:25.812251: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-02 09:13:26.703333: W tensorflow/compiler/tf2

In [3]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

2023-12-02 09:17:42.730425: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-02 09:17:42.803212: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-02 09:17:42.803715: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [4]:
# entity
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [5]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [6]:
class ConfigurationManager:
    """
    A class for managing configuration settings related to the components.

    """
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        prarms_filepath = PARAMS_FILE_PATH
        ) -> None:
        """
        Initializes the ConfigurationManager object.

        Args:
            config_filepath (str, optional): File path for the main configuration file. Defaults to CONFIG_FILE_PATH.
            params_filepath (str, optional): File path for the parameters file. Defaults to PARAMS_FILE_PATH.
        """
        
        self.config = read_yaml(config_filepath)
        self.prams = read_yaml(prarms_filepath)
        
        create_directories([self.config.artifacts_root])

    
    def get_validation_config(self) -> EvaluationConfig:
        """
        Returns an evaluation config data object.

        Returns:
            EvaluationConfig: Object containing the configuration for model evaluation.
        """
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chicken-fecal-images",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [7]:
from urllib.parse import urlparse

In [8]:
# component
class Evaluation:
    """
    A class for evaluating a trained model.

    Methods:
        __init__: Initializes the Evaluation object.
        _valid_generator: Generates data for validation and stores it in the valid generator.
        load_model: Loads a saved Keras model.
        evaluation: Evaluates the loaded model on the validation data and saves the scores.
        save_score: Saves the evaluation scores as a JSON file.

    Attributes:
        config (EvaluationConfig): Object containing the configuration for model evaluation.
        valid_generator: Data generator for validation data.
        model (tf.keras.Model): Loaded Keras model.
        score (list): Evaluation scores for loss and accuracy.

    """
    def __init__(self, config: EvaluationConfig):
        """
        Initializes the Evaluation object.

        Args:
            config (EvaluationConfig): Object containing the configuration for model evaluation.
        """
        self.config = config

    
    def _valid_generator(self):
        """
        Generate data for validation and store it to valid generator
        """
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        """
        Loads a pre-trained model.

        Args:
            path (Path): Path to the saved model file.

        Returns:
            tf.keras.Model: Loaded pre-trained model.
        """
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        """ 
        Evaluating the model and save it to score
        """ 
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)

    
    def save_score(self):
        """ 
        Saving score as a json file
        """
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

In [9]:
# pipeline
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()

except Exception as e:
   raise e

[2023-12-02 09:37:08,882: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-12-02 09:37:08,884: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-02 09:37:08,885: INFO: common: created directory at: artifacts]
Found 116 images belonging to 2 classes.


2023-12-02 09:37:09.467359: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2023-12-02 09:37:10.847490: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.54GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-12-02 09:37:11.746352: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.54GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-12-02 09:37:13.214729: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.30GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be

8/8 [==============================] - 13s 573ms/step - loss: 33.6786 - accuracy: 0.5000
[2023-12-02 09:37:21,985: INFO: common: json file saved at: scores.json]
